In [ ]:
import csv
import json
import pandas as pd
import re
# !pip install spacy textblob
# !python -m spacy download en_core_web_sm
import spacy
from textblob import TextBlob
import ast
import numpy as np
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_file_path = "/content/drive/MyDrive/Microaggressions_data/microaggressions.csv"
output_csv_file_path = "labelled_data.csv"

In [ ]:
# df = pd.read_csv(input_file_path)
selected_columns = ['summary', 'content']

df = pd.read_csv(input_file_path, usecols=selected_columns)
df.head()

,summary,content
0,"A meme blog posted a meme saying ""If deaf peop...","[{""text"":""A meme blog posted a meme saying \""I..."
1,I think black girls with short hair are ugly.,"[{""text"":""I think black girls with short hair ..."
2,"It Girl, a game on Facebook, requires that the...","[{""text"":""It Girl, a game on Facebook, require..."
3,"She’s SO Bipolar, haha!","[{""text"":""She's SO Bipolar, haha!""},{""text"":""C..."
4,Old woman: “Schools shouldn’t offer free lunch...,"[{""text"":""Old woman: “Schools shouldn’t offer ..."


In [ ]:
df.shape

(3599, 2)

In [ ]:
df_copy = df.copy()
print(df_copy['content'][21])
print(df_copy['summary'][21])

df_copy['content'] = df_copy['content'].explode()
df_copy['content'] = df_copy['content'].fillna('[]')
df_copy['content'] = df_copy['content'].apply(json.loads)
df_copy['content'][21]


nan
Being in a class the first day, when the professor is trying to learn names. He asks the names of everyone who speaks. I speak;...


[]

In [ ]:
def extract_text(entries):
    return ', '.join([entry.get('text', '') for entry in entries])

df_copy['summary'] = df_copy['content'].apply(extract_text)


In [ ]:
df_copy_filtered = df_copy[df_copy['summary'] != '']

# Create a new DataFrame from the list of dictionaries
df_filtered_text = pd.DataFrame(df_copy_filtered)

# Reset the index of the new DataFrame
df_filtered_text.reset_index(drop=True, inplace=True)

df_combined = pd.concat([df, df_filtered_text], ignore_index=True)


In [ ]:
df_combined.head()

,summary,content
0,"A meme blog posted a meme saying ""If deaf peop...","[{""text"":""A meme blog posted a meme saying \""I..."
1,I think black girls with short hair are ugly.,"[{""text"":""I think black girls with short hair ..."
2,"It Girl, a game on Facebook, requires that the...","[{""text"":""It Girl, a game on Facebook, require..."
3,"She’s SO Bipolar, haha!","[{""text"":""She's SO Bipolar, haha!""},{""text"":""C..."
4,Old woman: “Schools shouldn’t offer free lunch...,"[{""text"":""Old woman: “Schools shouldn’t offer ..."


In [ ]:
df_combined['Label'] = 'MICRO'


In [ ]:
def extract_text(content):
    try:
        content_list = ast.literal_eval(content)
        text_values = [item.get("text", "") for item in content_list]
        return " ".join(text_values)
    except (ValueError, SyntaxError):
        return ""


In [ ]:
df['full_content'] = df['content'].apply(extract_text)

In [ ]:
df['Label'] = 'MICRO'


In [ ]:
df.head()

,summary,content,Label
0,"A meme blog posted a meme saying ""If deaf peop...","[{""text"":""A meme blog posted a meme saying \""I...",MICRO
1,I think black girls with short hair are ugly.,"[{""text"":""I think black girls with short hair ...",MICRO
2,"It Girl, a game on Facebook, requires that the...","[{""text"":""It Girl, a game on Facebook, require...",MICRO
3,"She’s SO Bipolar, haha!","[{""text"":""She's SO Bipolar, haha!""},{""text"":""C...",MICRO
4,Old woman: “Schools shouldn’t offer free lunch...,"[{""text"":""Old woman: “Schools shouldn’t offer ...",MICRO


In [ ]:
df_combined.shape

(7198, 3)

In [ ]:
missing_count = df_combined.isnull().sum()
print(missing_count)

summary     2
content    86
Label       0
dtype: int64


In [ ]:
# Drop rows with missing values
df = df.dropna()
# df['tags'] = df['tags'].replace('nan', np.nan)

In [ ]:
print(df.isnull().sum())

summary    0
Label      0
dtype: int64


In [ ]:
df_combined.to_csv('/content/drive/MyDrive/Microaggressions_data/labelled_data.csv', index=False)


In [ ]:
df['tags'] = df['tags'].apply(eval)

filtered_rows = df[df['tags'].apply(lambda x: 'gender' in x)]

count = len(filtered_rows)

print("Filtered Rows:")
print(filtered_rows)
print("\nNumber of Rows:", count)

Filtered Rows:
                     date                           tags  \
1     2017-08-28 20:00:30       [xs, race, gender, body]   
5     2017-08-26 20:00:01                   [xs, gender]   
6     2017-08-26 12:00:30             [xs, race, gender]   
7     2017-08-25 20:00:10  [xs, sexuality, race, gender]   
9     2017-08-24 20:00:11        [xs, sexuality, gender]   
...                   ...                            ...   
3573  2010-11-30 05:11:29        [xs, gender, sexuality]   
3576  2010-11-30 04:25:34  [xs, ethnicity, race, gender]   
3577  2010-11-30 04:24:11                   [xs, gender]   
3594  2010-03-01 09:54:00                   [xs, gender]   
3598  2010-03-01 07:14:00             [xs, race, gender]   

                                                summary  
1         I think black girls with short hair are ugly.  
5     You’ve been married for two weeks! When are yo...  
6     At the office. Three Jamaican men with dreadlo...  
7                               

In [ ]:
# Convert string representations to actual lists
df['tags'] = df['tags'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else np.nan)

# Use the explode function to transform the lists into separate rows
df_exploded = df.explode('tags')

# Get unique tags
unique_tags_list = df_exploded['tags'].unique()

In [ ]:
unique_tags_list = unique_tags_list.tolist()

In [ ]:
unique_tags_list
# unique_tags_list.remove('nan')

['xs',
 'ableism',
 'race',
 'gender',
 'body',
 'sexuality',
 'neurodiversity',
 'class',
 'language',
 'ethnicity',
 'trans*',
 nan,
 'nationality',
 'internalization',
 'survivor',
 'sex',
 'trans',
 'age',
 'ability',
 'religion',
 'acces',
 'education',
 'immigration',
 'meta',
 'school',
 'ody',
 'mental health',
 'immigration status',
 '3yearsago',
 'reblog',
 'racism',
 'macro',
 'family',
 'hair',
 'beauty',
 'queer',
 'genderqueer',
 'marriage',
 'mind',
 'colonialism',
 'skin tone',
 'Skin Tone',
 'eurocentricism',
 'eurocentrism',
 'west',
 'immigrant status',
 'region',
 'orientalism',
 'marriagestatus',
 'Mind',
 'everything',
 'Sexuality',
 'Survivor',
 'discussion',
 'surivor',
 'invisible',
 'patriarchy',
 'citizenship',
 'institutional',
 'reflection',
 'invisibility',
 'sexism',
 'faq',
 'event',
 'resistance',
 'relationship']

In [ ]:
input_file_path = '/content/drive/MyDrive/interracialdating_comments/interracialdating_comments'
# input_file_path = "C:/Users/budhr/Downloads/reddit/subreddits/interracialwild_comments/interracialwild_comments"
# input_file_path = "C:/Users/budhr/Downloads/reddit/subreddits/alcoholicsanonymous_comments/alcoholicsanonymous_comments"
# input_file_path = "C:/Users/budhr/Downloads/reddit/subreddits/AutisticWithADHD_comments/AutisticWithADHD_comments"
output_file_path = "/content/drive/MyDrive/interracialdating_comments.csv"
# interracialdating_comments



In [ ]:

# # Load JSON data
with open(input_file_path,  encoding='utf-8') as f:
  json_data = f.read()

field_names = ["author","body", "score", "created_utc", "subreddit", "id", "controversiality"]
# ["created_utc", "author_flair_css_class", "score", "ups", "subreddit", "stickied", "link_id", "subreddit_id", "controversiality", "body", "retrieved_on", "distinguished", "gilded", "id", "parent_id", "edited", "author_flair_text", "author"]  # Modify as needed

with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:

    # Create CSV writer
    writer = csv.DictWriter(csvfile, fieldnames=field_names)

    # Write header row
    writer.writeheader()

    # Parse each JSON object
    for line in json_data.splitlines():

        # Load JSON
        data = json.loads(line)

        # Extract required fields
        row = {field: data[field] for field in field_names}

        # Write row to CSV
        writer.writerow(row)


In [ ]:

# Read CSV into DataFrame
df = pd.read_csv(output_file_path)

# Display the DataFrame
print(df.head())
df.shape

        author                                               body  score  \
0    Umbertkid  I've been with a lovely man for almost a year ...      6   
1   keakealani  Mixed Asian/Hawaiian here marrying a White-as-...      3   
2   d0nt_b1ink  Howdy! A month from tomorrow I will have been ...      5   
3  StopItLink4  Oh well it wasn't me. I believe someone on /r/...      5   
4  StopItLink4  I'm curious about the children aspect as well....      3   

   created_utc          subreddit       id  controversiality  
0   1366590604  interracialdating  c9jxqpd                 0  
1   1366592726  interracialdating  c9jyg0v                 0  
2   1366592983  interracialdating  c9jyj7m                 0  
3   1366595449  interracialdating  c9jzds0                 0  
4   1366595537  interracialdating  c9jzetj                 0  


(87354, 7)

In [ ]:
missing_values = df.isnull().sum()

print("Missing Values:\n", missing_values)

Missing Values:
 author              0
body                1
score               0
created_utc         0
subreddit           0
id                  0
controversiality    0
dtype: int64


In [ ]:
df = df.dropna()

In [ ]:
df.shape

(87353, 7)

In [ ]:
df.columns

Index(['author', 'body', 'score', 'created_utc', 'subreddit', 'id',
       'controversiality'],
      dtype='object')

In [ ]:
df["body"] = df["body"].apply(lambda x: str(x).lower())
df["body"] = df["body"].apply(lambda x: re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x))
df["body"] = df["body"].apply(lambda x: re.sub(r'[\n\r,\.]', ' ', x))



In [ ]:
nlp = spacy.load('en_core_web_sm')

def normalize_text(text):
    doc = nlp(text)
    normalized_text = " ".join(token.lemma_ for token in doc if not token.is_stop)
    return normalized_text

# Apply text normalization
df['normalized_body'] = df['body'].apply(normalize_text)


In [ ]:
df['normalized_body'].head()

0    lovely man year smitten . think mesh essential...
1    mix asianhawaiian marry white - - white - - ma...
2    howdy ! month tomorrow married husband year . ...
3          oh . believe blackladie credit credit . : d
4    curious child aspect . fit , etc etc . real bi...
Name: normalized_body, dtype: object

In [ ]:
def analyze_sentiment(text):
    analysis = TextBlob(text)
    return "Positive" if analysis.sentiment.polarity > 0 else "Negative" if analysis.sentiment.polarity < 0 else "Neutral"


In [ ]:
df['sentiment'] = df['normalized_body'].apply(analyze_sentiment)

print(df[['normalized_body', 'sentiment']])


                                         normalized_body sentiment
0      lovely man year smitten . think mesh essential...  Positive
1      mix asianhawaiian marry white - - white - - ma...  Positive
2      howdy ! month tomorrow married husband year . ...  Positive
3            oh . believe blackladie credit credit . : d   Neutral
4      curious child aspect . fit , etc etc . real bi...  Positive
...                                                  ...       ...
87349                                       say , true .  Positive
87350  idea mean , suggest stay away tik tok date tre...  Negative
87351                         racial purity reason think   Neutral
87352  time establish identity marriage ? insecurity ...  Positive
87353  lol post suggest confidence insecurity ! defin...  Positive

[87353 rows x 2 columns]


In [ ]:
sentiment_counts = df['sentiment'].value_counts()

print("Sentiment Counts: {}".format(sentiment_counts))


Sentiment Counts: Positive    41722
Neutral     28561
Negative    17070
Name: sentiment, dtype: int64


In [ ]:
df[df["sentiment"] =="Negative"]["normalized_body"]

15       date high school ,   current boyfriend reconne...
19       black background mention uk ? caribbean ? afri...
20       caribbean .    black background different ... ...
22       ahhh , .   guess obviously " african american ...
24       depend . hear sort base level parentage . , " ...
                               ...                        
87333                                         exactly mean
87341    ? ? comment totally ignorant side . want learn...
87345    family relationship complicated deserve relati...
87347              try weird like , , report let know go .
87350    idea mean , suggest stay away tik tok date tre...
Name: normalized_body, Length: 17070, dtype: object

In [ ]:
shared_folder_path = "/content/drive/MyDrive/Microaggressions_data/"
goEmotions_1 = pd.read_csv(shared_folder_path+"goemotions_1.csv")
goEmotions_2 = pd.read_csv(shared_folder_path+"goemotions_2.csv")
goEmotions_3 = pd.read_csv(shared_folder_path+"goemotions_3.csv")

print(goEmotions_1.columns)
print(goEmotions_2.columns)
print(goEmotions_3.columns)

Index(['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear', 'admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral'],
      dtype='object')
Index(['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear', 'admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 

In [ ]:
positive_tags = ['text','caring', 'admiration','gratitude','joy','love', 'optimism', 'pride']
negative_tags = ['text','anger', 'embarrassment','disappointment','anger','disgust','disapproval']

df_1_pos = goEmotions_1[positive_tags]
df_2_pos = goEmotions_2[positive_tags]
df_3_pos = goEmotions_3[positive_tags]

df_1_neg = goEmotions_1[negative_tags]
df_2_neg = goEmotions_2[negative_tags]
df_3_neg = goEmotions_3[negative_tags]

In [ ]:
pos_tags = ['caring', 'admiration','gratitude','joy','love', 'optimism', 'pride']
neg_tags = ['anger', 'embarrassment','disappointment','anger','disgust','disapproval']

In [ ]:
def sentences(df,tag):
  df_tag = df[df[tag].any(axis=1)]['text']
  return df_tag.values

In [ ]:
positive_sentences = sentences(df_1_pos,pos_tags)
negative_sentences = sentences(df_1_neg, neg_tags)

In [ ]:
len(positive_sentences), len(negative_sentences)

(17992, 10856)

In [ ]:
labelled_data = pd.read_csv(shared_folder_path+"labelled_data.csv")
labelled_data

,summary,Label
0,"A meme blog posted a meme saying ""If deaf peop...",MICRO
1,I think black girls with short hair are ugly.,MICRO
2,"It Girl, a game on Facebook, requires that the...",MICRO
3,"She’s SO Bipolar, haha!",MICRO
4,Old woman: “Schools shouldn’t offer free lunch...,MICRO
...,...,...
3593,I text a male friend to ask if he wants to see...,MICRO
3594,Swing dancing during orientation week of colle...,MICRO
3595,Made to hold the Chinese flag for grade school...,MICRO
3596,Calm down.,MICRO


In [ ]:
pos_df = pd.DataFrame({'summary':positive_sentences[0:4000], 'Label':'POS'})
neg_df = pd.DataFrame({'summary':negative_sentences[0:4000], 'Label':'NEG'})

In [ ]:
labelled_data = pd.concat([labelled_data, pos_df, neg_df]).reset_index(drop=True)
labelled_data.to_csv("/content/drive/MyDrive/Microaggressions_data/labelled_data.csv", index=False)

In [ ]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the pre-trained model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")
tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")

# Function to calculate toxicity score for a given text
def calculate_toxicity(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    toxicity_score = outputs.logits[0][1].item()
    return toxicity_score

# Apply the function to the 'text' column and create a new column 'toxicity_score'
df['toxicity_score'] = df['summary'].apply(calculate_toxicity)

# Display the DataFrame with the calculated toxicity scores
print(df)

                                                 summary  Label  \
0      A meme blog posted a meme saying "If deaf peop...  MICRO   
1          I think black girls with short hair are ugly.  MICRO   
2      It Girl, a game on Facebook, requires that the...  MICRO   
3                                She’s SO Bipolar, haha!  MICRO   
4      Old woman: “Schools shouldn’t offer free lunch...  MICRO   
...                                                  ...    ...   
11593                                      oops, my bad.    NEG   
11594        I guess not, thanks for clearing it up mate    NEG   
11595  Don’t spend time with someone who isn’t going ...    NEG   
11596      I know, all the comments have me cracking up.    NEG   
11597    Yes. You fucking troll me every time. Fuck you.    NEG   

       toxicity_score  
0           -7.996507  
1           -3.337136  
2           -9.244769  
3           -7.086105  
4           -8.797015  
...               ...  
11593       -8.906732  
115

In [ ]:
df.to_csv("/content/drive/MyDrive/Microaggressions_data/toxicity_score.csv", index=False)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Microaggressions_data/toxicity_score.csv")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW


In [ ]:
# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [ ]:
# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Assuming binary classification


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
df.head()

,summary,Label,toxicity_score
0,"A meme blog posted a meme saying ""If deaf peop...",MICRO,-7.996507
1,I think black girls with short hair are ugly.,MICRO,-3.337136
2,"It Girl, a game on Facebook, requires that the...",MICRO,-9.244769
3,"She’s SO Bipolar, haha!",MICRO,-7.086105
4,Old woman: “Schools shouldn’t offer free lunch...,MICRO,-8.797015


In [ ]:
import torch

In [ ]:
# Convert categorical labels to numerical labels using LabelEncoder
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['output_label'] = label_encoder.fit_transform(df['Label'])


In [ ]:
df.drop('Label',axis=1, inplace=True)

In [ ]:
# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['output_label'].unique()))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Create DataLoader for training and testing data
train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_inputs['toxicity_score'], train_labels)
test_dataset = TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], test_inputs['toxicity_score'], test_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Set up the model for training
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = CrossEntropyLoss()

TypeError: ignored

In [ ]:
# Evaluation on the test set
model.eval()
all_predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, toxicity_score, labels = batch
        logits = model(input_ids, attention_mask=attention_mask, toxicity_score=toxicity_score)
        predictions = torch.argmax(logits, dim=1)
        all_predictions.extend(predictions.numpy())

# Calculate accuracy
accuracy = accuracy_score(test_df['output_label'], all_predictions)
print(f"Accuracy: {accuracy}")

In [ ]:
# Extract text and toxicity score
train_text = train[['summary']]
train_labels = train[['output_label']]
train_toxicity = train[['output_label']]

test_text = test[['summary']]
test_labels = test[['output_label']]
test_toxicity = test[['toxicity_score']]


In [ ]:
train_text_df = pd.DataFrame(train['summary'].values, columns=['summary'])
train_toxicity = train['toxicity_score'].values

train_labels = train['output_label'].values

model = ClassificationModel('bert', 'bert-base-uncased', num_labels=5, use_cuda=False, args={'reprocess_input_data': True})

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##LIME Code

In [ ]:
# Install required libraries
# pip install transformers lime

from transformers import BertTokenizer, BertForSequenceClassification
from lime import lime_text
from lime.lime_text import LimeTextExplainer

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Define the function to predict using the BERT model
def predict_fn(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    return logits.detach().numpy()

# Example text for explanation
text_to_explain = "This is an example sentence for explanation."

# Create LIME explainer
explainer = LimeTextExplainer()

# Generate an explanation
exp = explainer.explain_instance(text_to_explain, predict_fn, num_features=6)

# Print the explanation
print('Explanation for the prediction:', exp.as_list())

##REST

In [ ]:
model.train_model(
    train_text_df,
    train_labels,
    args={
        'additional_features': train_toxicity
    }
)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


IndexError: ignored

In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer

# Get data
summary = df['summary'].values
toxicity = df['toxicity_score'].values
labels = df['output_label'].values

# Load model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize text
encoded_text = tokenizer(summary, padding=True, truncation=True, return_tensors='pt')

# Concatenate toxicity scores
encoded_data = {
    'input_ids': encoded_text['input_ids'],
    'attention_mask': encoded_text['attention_mask'],
    'toxicity': toxicity
}

# Create dataloader
dataloader = DataLoader(encoded_data, batch_size=16, shuffle=True)

# Optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: ignored

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertForSequenceClassification, BertTokenizer

# Assuming df is your DataFrame
summary = df['summary'].values
toxicity = df['toxicity_score'].values
labels = df['output_label'].values

# Load model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize text
encoded_text = tokenizer(summary, padding=True, truncation=True, return_tensors='pt')

# Convert labels to tensor
labels_tensor = torch.tensor(labels)

# Concatenate toxicity scores
input_ids = encoded_text['input_ids']
attention_mask = encoded_text['attention_mask']
toxicity_tensor = torch.tensor(toxicity).view(-1, 1)  # Assuming toxicity is a 1D array

# Combine all tensors into a single dataset
dataset = TensorDataset(input_ids, attention_mask, toxicity_tensor, labels_tensor)

# Create dataloader
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

# Now you can use the dataloader in your training loop
for epoch in range(num_epochs):
    model.train()
    for batch in dataloader:
        input_ids, attention_mask, toxicity, labels = batch

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, additional_feature=toxicity)
        logits = outputs.logits

        # Calculate loss
        loss = criterion(logits, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: ignored

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertForSequenceClassification, BertTokenizer
import pandas as pd

# Sample DataFrame
data = {'summary': ['This is a positive summary.', 'This is a negative summary.'],
        'toxicity_score': [0.8, 0.2],
        'output_label': [1, 0]}

df = pd.DataFrame(data)

# Get data
summary = df['summary'].values
toxicity = df['toxicity_score'].values
labels = df['output_label'].values

# Load model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize text
encoded_text = tokenizer(summary.tolist(), padding=True, truncation=True, return_tensors='pt')

# Convert labels to tensor
labels_tensor = torch.tensor(labels)

# Concatenate input tensors
input_ids = encoded_text['input_ids']
attention_mask = encoded_text['attention_mask']
toxicity_tensor = torch.tensor(toxicity).view(-1, 1)  # Assuming toxicity is a 1D array

# Combine all tensors into a single dataset
dataset = TensorDataset(input_ids, attention_mask, toxicity_tensor, labels_tensor)

# Create dataloader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Sample training loop
num_epochs = 1
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    for batch in dataloader:
        input_ids, attention_mask, toxicity, labels = batch

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Calculate loss
        loss = criterion(logits, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

print("Training completed.")


Training completed.


In [ ]:
df.shape

(11598, 3)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming 'df' is your original dataframe with three classes
# Separate the data by class
class_0 = df[df['output_label'] == 0]
class_1 = df[df['output_label'] == 1]
class_2 = df[df['output_label'] == 2]

# Set the desired number of instances for each class
desired_instances_per_class = 500  # Assuming an equal split among classes

# Randomly sample instances from each class
sampled_class_0 = class_0.sample(desired_instances_per_class)
sampled_class_1 = class_1.sample(desired_instances_per_class)
sampled_class_2 = class_2.sample(desired_instances_per_class)

# Combine the sampled subsets to create the reduced dataset
reduced_dataset = pd.concat([sampled_class_0, sampled_class_1, sampled_class_2])

# Optional: Shuffle the rows to randomize the order
reduced_dataset = reduced_dataset.sample(frac=1).reset_index(drop=True)

# Display the reduced dataset
print(reduced_dataset)

                                                summary  toxicity_score  \
0     I like you, you’re cool. But some others, you ...       -8.925307   
1     Yeah I know, it’s just wishful thinking but I ...       -8.905490   
2                   Married to the love of my life. LOL       -9.273452   
3                    Thank you, how epic is your curl?        -9.031632   
4     too bad for [NAME] we don't negotiate with ter...       -7.976358   
...                                                 ...             ...   
1495                                       No. Just no.       -9.183431   
1496              Stop saying generic hating sentences!       -7.208015   
1497                                 great pass wiggins       -9.014302   
1498                                            “Cute.”       -9.116370   
1499  That must be it? Or that people buy carbon dio...       -9.224232   

      output_label  
0                0  
1                2  
2                2  
3              

In [ ]:
df.head()
#import torch


,summary,toxicity_score,output_label
0,"A meme blog posted a meme saying ""If deaf peop...",-7.996507,0
1,I think black girls with short hair are ugly.,-3.337136,0
2,"It Girl, a game on Facebook, requires that the...",-9.244769,0
3,"She’s SO Bipolar, haha!",-7.086105,0
4,Old woman: “Schools shouldn’t offer free lunch...,-8.797015,0


In [ ]:
df_train, df_test = train_test_split(reduced_dataset, test_size=0.2)
df_train, df_val = train_test_split(df_train, test_size=0.2)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text_data = df_train['summary'].values.tolist()

encoded_train = tokenizer(text_data, truncation=True, padding=True)
encoded_val = tokenizer(df_val['summary'].values.tolist() , truncation=True, padding=True)
encoded_test = tokenizer(df_test['summary'].values.tolist() , truncation=True, padding=True)

train_toxicity = torch.tensor(df_train['toxicity_score'].values)
val_toxicity = torch.tensor(df_val['toxicity_score'].values)
test_toxicity = torch.tensor(df_test['toxicity_score'].values)

df['output_label'] = df['output_label'].astype(int)
train_labels = torch.tensor(df_train['output_label'].values)
val_labels = torch.tensor(df_val['output_label'].values)
test_labels = torch.tensor(df_test['output_label'].values)


In [ ]:
# Model and dataloaders
num_labels = len(set(reduced_dataset['output_label'].unique()))
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

input_ids = torch.tensor(encoded_train['input_ids'])
attention_mask = torch.tensor(encoded_train['attention_mask'])

train_dataset = TensorDataset(input_ids, attention_mask, train_toxicity, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
input_ids = torch.tensor(encoded_val['input_ids'])
attention_mask = torch.tensor(encoded_val['attention_mask'])

val_dataset = TensorDataset(input_ids, attention_mask, val_toxicity, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=16)

In [ ]:
# Training loop
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
num_epochs = 1


def evaluate(model, dataloader):
  model.eval()

  correct = 0
  total = 0

  with torch.no_grad():
    for batch in dataloader:
      outputs = model(input_ids=batch[0], attention_mask=batch[1])
      predictions = torch.argmax(outputs, dim=1)
      correct += (predictions == batch[3]).sum()
      total += len(batch[3])

  return correct/total

In [ ]:
for epoch in range(num_epochs):
  model.train()

  for batch in train_dataloader:
    optimizer.zero_grad()
    loss = model(input_ids=batch[0], attention_mask=batch[1], labels=batch[3]).loss
    loss.backward()
    optimizer.step()

  val_acc = evaluate(model, val_dataloader)
  print(f"Epoch {epoch+1} - Val Acc: {val_acc}")

# Evaluation
test_acc = evaluate(model, encoded_test, test_toxicity, test_labels)
print(f"Test Acc: {test_acc}")

TypeError: ignored